<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
aux = """[{"\"Unnamed: 0\"":"1","\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"88.0","MONTH":"1","\"Presión (hPa)\"":"1028.5","HOUR":"0","\"Tendencia (hPa)\"":"-1.2","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"24.0","\"Estación\"":"Alpedrete","DAY":"28","\"Precipitación (mm)\"":"0.5","\"Temperatura (ºC)\"":"60.2","\"Velocidad del viento (km/h)\"":"34.6"}]"""
data = [aux.replace('{""','{"').replace(',""',',"').replace('"":','":')]

In [2]:
data=[{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Sariñena, Depósito agua","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Seira","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Sopeira","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"65","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"22","\"Estación\"":"Sos del Rey Católico","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.1","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tamarite de Litera","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""}]

In [3]:
import pandas as pd
new_observation=pd.json_normalize(data)
new_observation = new_observation.replace(r'^\s*$', np.nan, regex=True)

In [4]:

import pandas as pd
import joblib
import pickle
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
import pathlib

In [9]:
def model_selection(new_observation):
        #Preprocessing
    new_observation.columns=['province', 'dir_wind_streak','humidity','month','pressure','hour','tendency','year','wind_direction',
            'wind_streak','station','day','precipitation','temperature','wind_speed']
    
    
    #New data
    if (new_observation['temperature'].isnull().values.all(axis=0) | new_observation['humidity'].isnull().values.all(axis=0) | new_observation['pressure'].isnull().values.all(axis=0)):
        return
     #   return print('Too many missing values')
    else:
        #Preprocessing
        new_observation=new_observation[new_observation['temperature'].notna()]
        new_observation=new_observation[new_observation['pressure'].notna()]   
        new_observation=new_observation[new_observation['humidity'].notna()]   
        
        new_observation['precipitation'].fillna(0, inplace=True)
        new_observation['wind_speed'].fillna(0, inplace=True)
        new_observation["wind_direction"].fillna("Calma", inplace = True)
        new_observation_cleaned=new_observation.drop(columns=['dir_wind_streak', 'month','hour', 'tendency', 'year', 'wind_direction', 'wind_streak', 'station',
       'day'])
    
        target_province=list(new_observation['province'].unique())
    
    return target_province

In [10]:
def process_newobservation(new_observation, scaler, target_province):
    
    if target_province is None:
        return
    
    else:
        new_observation=new_observation[new_observation['temperature'].notna()]
        new_observation=new_observation[new_observation['pressure'].notna()]   
        new_observation=new_observation[new_observation['humidity'].notna()]   
        
        new_observation['precipitation'].fillna(0, inplace=True)
        new_observation['wind_speed'].fillna(0, inplace=True)
        new_observation["wind_direction"].fillna("Calma", inplace = True)
        new_observation_cleaned=new_observation.drop(columns=['dir_wind_streak', 'month','hour', 'tendency', 'year', 'wind_direction', 'wind_streak', 'station',
       'day'])
    
    
        df_provinces={}

        for i in target_province:
            df_provinces[i]=new_observation_cleaned.loc[new_observation_cleaned['province'] == i]
  
        #Remove categorical variables
        X_provinces={}

        for name, df in df_provinces.items():
            X=df.drop(columns=['province'])
            X_provinces.update({name : X})
        
        #Scaling - Transform
        X_provinces_scaled={}

        for name, df in X_provinces.items():
            scale=scaler[name].transform(X_provinces[name])
            X_provinces_scaled.update({name : scale})
            X_provinces_scaled[name]=pd.DataFrame(X_provinces_scaled[name], columns=X_provinces[name].columns)

        
        return X_provinces_scaled

In [11]:
def check_outliers(X_new,model,mapper,province):
    '''
    This function checks if there is outliers or not in the newdataset
    '''
    if X_new is None:
        return
    else:
        new_provinces={}
    
        for name, df in X_new.items():
    
    #Take the new distance matrix and the new minimum distance
            new_labels = model[name].predict(X_new[name])
            new_distance_matrix=pd.DataFrame(model[name].transform(X_new[name]))
            new_min_distance=[]
    
            for index, row in new_distance_matrix.iterrows():
                mindist=new_distance_matrix.iloc[index].min()
                new_min_distance.append(mindist)
    
    #Create a dataframe with the new values for cluster, distance, outliers, thresholds...
            new_distances_clusters=pd.DataFrame(columns=['cluster', 'distance'])
            new_distances_clusters['cluster']=new_labels
            new_distances_clusters['distance']=new_min_distance
    
    #Apply the mapper to assign the threshold for each cluster
            new_distances_clusters['threshold'] = new_distances_clusters['cluster'].map(mapper[name])
    
            new_distances_clusters['Anomaly']=0
    
    #Create anomaly column
            for index, row in new_distances_clusters.iterrows():
                if (new_distances_clusters['distance'][index] >= new_distances_clusters['threshold'][index]):
                    new_distances_clusters['Anomaly'][index]=1
                if (new_distances_clusters['distance'][index] <= new_distances_clusters['threshold'][index]):
                    new_distances_clusters['Anomaly'][index]=0
    
            print('Number of anomalies in {}: {}:'.format(name,np.count_nonzero(new_distances_clusters['Anomaly'] == 1)))
            new_provinces.update({name: new_distances_clusters})
    
    return new_provinces

In [12]:
with open('minmax_provinces_kmeans_last.pickle', 'rb') as handle:
        scaler_provinces = pickle.load(handle)


# Load models (deserialize)
with open('model_provinces_kmeans_last.pickle', 'rb') as handle:
        models_provinces = pickle.load(handle)
    

    # Load mapper (deserialize)
with open('mapper_provinces_kmeans_last.pickle', 'rb') as handle:
        mapper_provinces = pickle.load(handle)


In [41]:
 province=model_selection(new_observation)

In [51]:
new_clean=process_newobservation(new_observation, scaler_provinces, target_province=province)

In [56]:
distances=check_outliers(new_clean,models_provinces,mapper_provinces,province=province)

In [5]:
new_data=[{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"86","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"9","\"Estación\"":"Alcala de Henares","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.5","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Alpedrete","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"9","\"Estación\"":"Aranjuez","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.4","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Arganda del Rey","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"66","MONTH":"3","\"Presión (hPa)\"":"901.4","HOUR":"10","\"Tendencia (hPa)\"":"0.9","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"8","\"Estación\"":"Buitrago del Lozoya","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.4","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"75","MONTH":"3","\"Presión (hPa)\"":"902.3","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"9","\"Estación\"":"Colmenar Viejo","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.3","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"59","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"4","\"Estación\"":"Madrid, Ciudad Universitaria","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.4","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Madrid, El Goloso","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"62","MONTH":"3","\"Presión (hPa)\"":"938.7","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"10","\"Estación\"":"Madrid, Retiro","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.7","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"83","MONTH":"3","\"Presión (hPa)\"":"949.7","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"11","\"Estación\"":"Madrid Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.7","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Pozuelo de Alarcón","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.7","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"68","MONTH":"3","\"Presión (hPa)\"":"846.1","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"26","\"Estación\"":"Puerto Alto del León","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"0.2","\"Velocidad del viento (km/h)\"":"17"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"62","MONTH":"3","\"Presión (hPa)\"":"809.5","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"26","\"Estación\"":"Puerto de Navacerrada","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"0.1","\"Velocidad del viento (km/h)\"":"10"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Rascafría","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"54","MONTH":"3","\"Presión (hPa)\"":"928.8","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"8","\"Estación\"":"Robledo de Chavela","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.8","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Rozas de Puerto Real","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"San Sebastián de Los Reyes","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"57","MONTH":"3","\"Presión (hPa)\"":"853.9","HOUR":"10","\"Tendencia (hPa)\"":"0.7","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Somosierra","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"3.0","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tielmes","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"67","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"9","\"Estación\"":"Torrelodones","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.0","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"76","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"9","\"Estación\"":"Valdemorillo","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.6","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"942.0","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"11","\"Estación\"":"Villanueva de la Cañada","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.5","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"86","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"9","\"Estación\"":"Alcala de Henares","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.5","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Alpedrete","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"9","\"Estación\"":"Aranjuez","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.4","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Arganda del Rey","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"66","MONTH":"3","\"Presión (hPa)\"":"901.4","HOUR":"10","\"Tendencia (hPa)\"":"0.9","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"8","\"Estación\"":"Buitrago del Lozoya","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.4","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"75","MONTH":"3","\"Presión (hPa)\"":"902.3","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"9","\"Estación\"":"Colmenar Viejo","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.3","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"59","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"4","\"Estación\"":"Madrid, Ciudad Universitaria","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.4","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Madrid, El Goloso","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"62","MONTH":"3","\"Presión (hPa)\"":"938.7","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"10","\"Estación\"":"Madrid, Retiro","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.7","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"83","MONTH":"3","\"Presión (hPa)\"":"949.7","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"11","\"Estación\"":"Madrid Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.7","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Pozuelo de Alarcón","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.7","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"68","MONTH":"3","\"Presión (hPa)\"":"846.1","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"26","\"Estación\"":"Puerto Alto del León","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"0.2","\"Velocidad del viento (km/h)\"":"17"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"62","MONTH":"3","\"Presión (hPa)\"":"809.5","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"26","\"Estación\"":"Puerto de Navacerrada","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"0.1","\"Velocidad del viento (km/h)\"":"10"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Rascafría","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"54","MONTH":"3","\"Presión (hPa)\"":"928.8","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"8","\"Estación\"":"Robledo de Chavela","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.8","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Rozas de Puerto Real","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"San Sebastián de Los Reyes","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"57","MONTH":"3","\"Presión (hPa)\"":"853.9","HOUR":"10","\"Tendencia (hPa)\"":"0.7","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Somosierra","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"3.0","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tielmes","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"67","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"9","\"Estación\"":"Torrelodones","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.0","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"76","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"9","\"Estación\"":"Valdemorillo","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.6","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"942.0","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"11","\"Estación\"":"Villanueva de la Cañada","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.5","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Alforja","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"57","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"10","\"Estación\"":"Arenys de Mar","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"15.2","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"88","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Artesa de Segre","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.9","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Balsareny","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"80","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"14","\"Estación\"":"Barcelona","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.3","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Barcelona, Museo Marítimo","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"72","MONTH":"3","\"Presión (hPa)\"":"1015.3","HOUR":"10","\"Tendencia (hPa)\"":"2.0","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"24","\"Estación\"":"Barcelona Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.9","\"Velocidad del viento (km/h)\"":"17"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Berga","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Blanes","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Bossòst","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cabacés","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cabo de Creus","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Caldes de Montbui","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Castell, Platja d'Aro","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Castelló d'Empúries","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Coll de Nargó","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Corbera, Pic d'Agulles","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"El Soleràs","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Espolla","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Esterri d'Àneu","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"54","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"21","\"Estación\"":"Figueres","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"14.0","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"83","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Fogars de Montclús","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.6","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Girona","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"85","MONTH":"3","\"Presión (hPa)\"":"999.3","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"11","\"Estación\"":"Girona Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.2","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Granyanella","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Horta de Sant Joan","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"82","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"18","\"Estación\"":"Igualada","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.4","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"L'Estartit","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"64","MONTH":"3","\"Presión (hPa)\"":"826.1","HOUR":"10","\"Tendencia (hPa)\"":"0.8","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"13","\"Estación\"":"La Molina","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"2.6","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"80","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"La Pobla de Cérvoles","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.6","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"La Pobla de Massaluca","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"82","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"4","\"Estación\"":"La Seu d'Urgell","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.2","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"100","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"6","\"Estación\"":"La Vall de Bianya","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.1","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"100","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"","\"Estación\"":"Les Planes d'Hostoles","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.5","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"78","MONTH":"3","\"Presión (hPa)\"":"993.5","HOUR":"10","\"Tendencia (hPa)\"":"2.0","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"24","\"Estación\"":"Lleida","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.9","\"Velocidad del viento (km/h)\"":"15"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Llimiana","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"92","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"26","\"Estación\"":"Llorac","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.2","\"Velocidad del viento (km/h)\"":"19"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Maçanet de Cabrenys","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"86","MONTH":"3","\"Presión (hPa)\"":"981.5","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"13","\"Estación\"":"Manresa","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.4","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"89","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"5","\"Estación\"":"Martinet","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"2.4","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"96","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"22","\"Estación\"":"Moià","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.6","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Mollerussa","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"98","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Monistrol de Montserrat","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.9","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"87","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"8","\"Estación\"":"Naut Aran, Arties","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"3.7","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"100","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"13","\"Estación\"":"Os de Balaguer","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.9","\"Velocidad del viento (km/h)\"":"8"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"65","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"7","\"Estación\"":"Planoles","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.2","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Pontons","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"77","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"7","\"Estación\"":"Porqueres","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.2","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"86","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"17","\"Estación\"":"Prats de Lluçanès","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.1","\"Velocidad del viento (km/h)\"":"10"},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"61","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"30","\"Estación\"":"Rasquera","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.7","\"Velocidad del viento (km/h)\"":"15"},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"64","MONTH":"3","\"Presión (hPa)\"":"1006.7","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"33","\"Estación\"":"Reus Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.2","\"Velocidad del viento (km/h)\"":"21"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"","\"Estación\"":"Ripoll","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"3.0","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"58","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"14","\"Estación\"":"Santa Susanna","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.5","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Sant Hilari","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"98","MONTH":"3","\"Presión (hPa)\"":"947.7","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"7","\"Estación\"":"Sant Julià  de Vilatorta","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"4.4","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Sant Pau de Segúries","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Sitges","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"100","MONTH":"3","\"Presión (hPa)\"":"922.3","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"8","\"Estación\"":"Talarn","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"4.3","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tarragona","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"88","MONTH":"3","\"Presión (hPa)\"":"945.1","HOUR":"10","\"Tendencia (hPa)\"":"1.2","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"9","\"Estación\"":"Tona","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.1","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Torà","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"82","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Torre de Cabdella","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"1.3","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"62","MONTH":"3","\"Presión (hPa)\"":"1011.8","HOUR":"10","\"Tendencia (hPa)\"":"1.9","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"31","\"Estación\"":"Tortosa","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.9","\"Velocidad del viento (km/h)\"":"13"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tuixent","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Vall de Boí","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"71","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Valls","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.9","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Vilafranca del Penedès","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Vilassar de Dalt","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"86","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"9","\"Estación\"":"Alcala de Henares","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.5","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Alpedrete","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"9","\"Estación\"":"Aranjuez","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.4","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Arganda del Rey","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"66","MONTH":"3","\"Presión (hPa)\"":"901.4","HOUR":"10","\"Tendencia (hPa)\"":"0.9","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"8","\"Estación\"":"Buitrago del Lozoya","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.4","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"75","MONTH":"3","\"Presión (hPa)\"":"902.3","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"9","\"Estación\"":"Colmenar Viejo","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.3","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"59","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"4","\"Estación\"":"Madrid, Ciudad Universitaria","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.4","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Madrid, El Goloso","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"62","MONTH":"3","\"Presión (hPa)\"":"938.7","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"10","\"Estación\"":"Madrid, Retiro","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.7","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"83","MONTH":"3","\"Presión (hPa)\"":"949.7","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"11","\"Estación\"":"Madrid Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.7","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Pozuelo de Alarcón","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.7","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"68","MONTH":"3","\"Presión (hPa)\"":"846.1","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"26","\"Estación\"":"Puerto Alto del León","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"0.2","\"Velocidad del viento (km/h)\"":"17"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"62","MONTH":"3","\"Presión (hPa)\"":"809.5","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"26","\"Estación\"":"Puerto de Navacerrada","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"0.1","\"Velocidad del viento (km/h)\"":"10"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Rascafría","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"54","MONTH":"3","\"Presión (hPa)\"":"928.8","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"8","\"Estación\"":"Robledo de Chavela","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.8","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Rozas de Puerto Real","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"San Sebastián de Los Reyes","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"57","MONTH":"3","\"Presión (hPa)\"":"853.9","HOUR":"10","\"Tendencia (hPa)\"":"0.7","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Somosierra","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"3.0","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tielmes","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"67","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"9","\"Estación\"":"Torrelodones","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.0","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"76","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"9","\"Estación\"":"Valdemorillo","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.6","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"942.0","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"11","\"Estación\"":"Villanueva de la Cañada","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.5","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Alforja","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"57","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"10","\"Estación\"":"Arenys de Mar","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"15.2","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"88","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Artesa de Segre","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.9","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Balsareny","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"80","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"14","\"Estación\"":"Barcelona","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.3","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Barcelona, Museo Marítimo","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"72","MONTH":"3","\"Presión (hPa)\"":"1015.3","HOUR":"10","\"Tendencia (hPa)\"":"2.0","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"24","\"Estación\"":"Barcelona Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.9","\"Velocidad del viento (km/h)\"":"17"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Berga","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Blanes","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Bossòst","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cabacés","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cabo de Creus","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Caldes de Montbui","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Castell, Platja d'Aro","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Castelló d'Empúries","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Coll de Nargó","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Corbera, Pic d'Agulles","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"El Soleràs","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Espolla","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Esterri d'Àneu","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"54","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"21","\"Estación\"":"Figueres","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"14.0","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"83","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Fogars de Montclús","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.6","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Girona","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"85","MONTH":"3","\"Presión (hPa)\"":"999.3","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"11","\"Estación\"":"Girona Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.2","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Granyanella","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Horta de Sant Joan","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"82","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"18","\"Estación\"":"Igualada","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.4","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"L'Estartit","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"64","MONTH":"3","\"Presión (hPa)\"":"826.1","HOUR":"10","\"Tendencia (hPa)\"":"0.8","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"13","\"Estación\"":"La Molina","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"2.6","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"80","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"La Pobla de Cérvoles","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.6","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"La Pobla de Massaluca","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"82","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"4","\"Estación\"":"La Seu d'Urgell","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.2","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"100","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"6","\"Estación\"":"La Vall de Bianya","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.1","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"100","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"","\"Estación\"":"Les Planes d'Hostoles","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.5","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"78","MONTH":"3","\"Presión (hPa)\"":"993.5","HOUR":"10","\"Tendencia (hPa)\"":"2.0","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"24","\"Estación\"":"Lleida","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.9","\"Velocidad del viento (km/h)\"":"15"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Llimiana","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"92","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"26","\"Estación\"":"Llorac","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.2","\"Velocidad del viento (km/h)\"":"19"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Maçanet de Cabrenys","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"86","MONTH":"3","\"Presión (hPa)\"":"981.5","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"13","\"Estación\"":"Manresa","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.4","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"89","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"5","\"Estación\"":"Martinet","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"2.4","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"96","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"22","\"Estación\"":"Moià","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.6","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Mollerussa","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"98","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Monistrol de Montserrat","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.9","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"87","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"8","\"Estación\"":"Naut Aran, Arties","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"3.7","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"100","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"13","\"Estación\"":"Os de Balaguer","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.9","\"Velocidad del viento (km/h)\"":"8"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"65","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"7","\"Estación\"":"Planoles","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.2","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Pontons","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"77","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"7","\"Estación\"":"Porqueres","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.2","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"86","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"17","\"Estación\"":"Prats de Lluçanès","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.1","\"Velocidad del viento (km/h)\"":"10"},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"61","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"30","\"Estación\"":"Rasquera","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.7","\"Velocidad del viento (km/h)\"":"15"},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"64","MONTH":"3","\"Presión (hPa)\"":"1006.7","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"33","\"Estación\"":"Reus Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.2","\"Velocidad del viento (km/h)\"":"21"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"","\"Estación\"":"Ripoll","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"3.0","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"58","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"14","\"Estación\"":"Santa Susanna","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.5","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Sant Hilari","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"98","MONTH":"3","\"Presión (hPa)\"":"947.7","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"7","\"Estación\"":"Sant Julià  de Vilatorta","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"4.4","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Sant Pau de Segúries","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Sitges","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"100","MONTH":"3","\"Presión (hPa)\"":"922.3","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"8","\"Estación\"":"Talarn","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"4.3","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tarragona","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"88","MONTH":"3","\"Presión (hPa)\"":"945.1","HOUR":"10","\"Tendencia (hPa)\"":"1.2","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"9","\"Estación\"":"Tona","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.1","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Torà","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"82","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Torre de Cabdella","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"1.3","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"62","MONTH":"3","\"Presión (hPa)\"":"1011.8","HOUR":"10","\"Tendencia (hPa)\"":"1.9","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"31","\"Estación\"":"Tortosa","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.9","\"Velocidad del viento (km/h)\"":"13"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tuixent","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Vall de Boí","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"71","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Valls","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.9","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Vilafranca del Penedès","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Vilassar de Dalt","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"64","MONTH":"3","\"Presión (hPa)\"":"916.2","HOUR":"10","\"Tendencia (hPa)\"":"1.4","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"10","\"Estación\"":"Abla","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.1","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"49","MONTH":"3","\"Presión (hPa)\"":"1016.6","HOUR":"10","\"Tendencia (hPa)\"":"0.6","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"14","\"Estación\"":"Adra","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"16.1","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Aguilar de la Frontera","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"63","MONTH":"3","\"Presión (hPa)\"":"949.1","HOUR":"10","\"Tendencia (hPa)\"":"1.4","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"5","\"Estación\"":"Alajar","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"13.4","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Albox","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Alcalá La Real","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Algarrobo","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Almadén de la Plata","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"59","MONTH":"3","\"Presión (hPa)\"":"1014.0","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"13","\"Estación\"":"Almería Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"14.9","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"97","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"12","\"Estación\"":"Almonte","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.6","\"Velocidad del viento (km/h)\"":"8"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Álora","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"92","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"14","\"Estación\"":"Alosno, Tharsis","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.6","\"Velocidad del viento (km/h)\"":"8"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Alpandeire","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"99","MONTH":"3","\"Presión (hPa)\"":"992.7","HOUR":"10","\"Tendencia (hPa)\"":"1.1","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"8","\"Estación\"":"Andújar","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.0","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"88","MONTH":"3","\"Presión (hPa)\"":"968.0","HOUR":"10","\"Tendencia (hPa)\"":"1.4","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"7","\"Estación\"":"Antequera","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.9","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Aroche","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"67","MONTH":"3","\"Presión (hPa)\"":"1015.2","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"11","\"Estación\"":"Ayamonte","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"13.2","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"100","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"9","\"Estación\"":"Baeza","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.6","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Barbate","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"97","MONTH":"3","\"Presión (hPa)\"":"926.2","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"10","\"Estación\"":"Baza","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.7","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Benahavís","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Benamejí","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cabo de Gata","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"86","MONTH":"3","\"Presión (hPa)\"":"1016.4","HOUR":"10","\"Tendencia (hPa)\"":"1.4","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"11","\"Estación\"":"Cádiz","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"14.7","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cala","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cardeña","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Carmona","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Carrión de los Céspedes","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cartaya","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Castell de Ferro","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"80","MONTH":"3","\"Presión (hPa)\"":"952.0","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"8","\"Estación\"":"Cazalla de la Sierra","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.9","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cazorla","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Chipiona","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Coín","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"98","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"10","\"Estación\"":"Córdoba, Embalse de Guadanuño","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.5","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"96","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"13","\"Estación\"":"Córdoba, Prágdena","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.0","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"92","MONTH":"3","\"Presión (hPa)\"":"1006.7","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"24","\"Estación\"":"Córdoba Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.7","\"Velocidad del viento (km/h)\"":"12"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"93","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"6","\"Estación\"":"Cortes de la Frontera","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.9","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Dólar","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.9","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"81","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"12","\"Estación\"":"Doña Mencía","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.8","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Écija","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"El Bosque","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"76","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"7","\"Estación\"":"El Campillo","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.8","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"El Cerro de Andévalo","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"El Granado","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Espiel","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"62","MONTH":"3","\"Presión (hPa)\"":"1015.2","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"6","\"Estación\"":"Estepona","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"14.7","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"75","MONTH":"3","\"Presión (hPa)\"":"1014.6","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"19","\"Estación\"":"Fuengirola","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Fuente de Piedra","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Fuente Palmera","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Fuentes de Andalucía","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"75","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"10","\"Estación\"":"Garrucha, Puerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"13.6","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"93","MONTH":"3","\"Presión (hPa)\"":"950.6","HOUR":"10","\"Tendencia (hPa)\"":"1.1","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"5","\"Estación\"":"Granada Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.9","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"80","MONTH":"3","\"Presión (hPa)\"":"936.8","HOUR":"10","\"Tendencia (hPa)\"":"1.0","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"9","\"Estación\"":"Granada Base Aérea","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.7","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Granada-Cartuja","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"72","MONTH":"3","\"Presión (hPa)\"":"911.8","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"10","\"Estación\"":"Grazalema","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.7","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"83","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"10","\"Estación\"":"Guadalcanal","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.5","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"80","MONTH":"3","\"Presión (hPa)\"":"952.3","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"5","\"Estación\"":"Hinojosa del Duque","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.2","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"83","MONTH":"3","\"Presión (hPa)\"":"1014.3","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"10","\"Estación\"":"Huelva, Ronda Este","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.1","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"59","MONTH":"3","\"Presión (hPa)\"":"981.4","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"5","\"Estación\"":"Huércal-Overa","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.9","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"84","MONTH":"3","\"Presión (hPa)\"":"949.1","HOUR":"10","\"Tendencia (hPa)\"":"0.9","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"9","\"Estación\"":"Jaén","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.7","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"92","MONTH":"3","\"Presión (hPa)\"":"1013.3","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"9","\"Estación\"":"Jerez de la Frontera Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.0","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Jimena de la Frontera","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Lanjarón","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"99","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"6","\"Estación\"":"La Puebla de los Infantes","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.1","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"La Rambla","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"88","MONTH":"3","\"Presión (hPa)\"":"967.7","HOUR":"10","\"Tendencia (hPa)\"":"1.1","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"9","\"Estación\"":"La Roda de Andalucía","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.3","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Las Cabezas de San Juan","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Láujar de Andarax","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Linares","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Lora de Estepa","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"76","MONTH":"3","\"Presión (hPa)\"":"1011.1","HOUR":"10","\"Tendencia (hPa)\"":"1.2","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"20","\"Estación\"":"Málaga, Centro Meteorológico","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.5","\"Velocidad del viento (km/h)\"":"11"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"50","MONTH":"3","\"Presión (hPa)\"":"1013.0","HOUR":"10","\"Tendencia (hPa)\"":"0.6","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"16","\"Estación\"":"Málaga, Puerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"16.9","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"69","MONTH":"3","\"Presión (hPa)\"":"1015.5","HOUR":"10","\"Tendencia (hPa)\"":"1.4","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"32","\"Estación\"":"Málaga Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"13.7","\"Velocidad del viento (km/h)\"":"19"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Manilva","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"23","\"Estación\"":"Marbella","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"14.2","\"Velocidad del viento (km/h)\"":"10"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Marbella, Puerto","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Medina Sidonia","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"75","MONTH":"3","\"Presión (hPa)\"":"1011.8","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"17","\"Estación\"":"Moguer, El Arenosillo","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"13.4","\"Velocidad del viento (km/h)\"":"9"},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Montoro","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"82","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"7","\"Estación\"":"Morón de la Frontera","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.7","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Motril, Puerto","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Nerja","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Olvera","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Osuna","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Priego de Córdoba","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Rincón de la Victoria","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Ronda Instituto","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"86","MONTH":"3","\"Presión (hPa)\"":"1016.0","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"12","\"Estación\"":"Roquetas de Mar","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.4","\"Velocidad del viento (km/h)\"":"8"},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Rota, Base Naval","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Salobreña","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"92","MONTH":"3","\"Presión (hPa)\"":"1012.7","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"14","\"Estación\"":"San Fernando","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"13.8","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"98","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"6","\"Estación\"":"San José del Valle","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.7","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"San Roque","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"91","MONTH":"3","\"Presión (hPa)\"":"927.8","HOUR":"10","\"Tendencia (hPa)\"":"1.2","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"12","\"Estación\"":"Santa Elena","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.9","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"99","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"9","\"Estación\"":"Sevilla, Tablada","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.5","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"97","MONTH":"3","\"Presión (hPa)\"":"1013.1","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"13","\"Estación\"":"Sevilla Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.6","\"Velocidad del viento (km/h)\"":"10"},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tarifa","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"91","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"9","\"Estación\"":"Tomares, Zaudín","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.3","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Torremolinos","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Torres","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Válor","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Valsequillo","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"","\"Humedad (%)\"":"90","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Valverde del Camino","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.1","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"84","MONTH":"3","\"Presión (hPa)\"":"994.8","HOUR":"10","\"Tendencia (hPa)\"":"1.4","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"14","\"Estación\"":"Vejer de la Frontera","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"13.7","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Vélez Blanco - Topares","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Vélez-Málaga","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"67","MONTH":"3","\"Presión (hPa)\"":"929.9","HOUR":"10","\"Tendencia (hPa)\"":"1.2","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"13","\"Estación\"":"Villanueva de Córdoba","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.1","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Villanueva del Arzobispo","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Villarrasa","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Víznar","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"86","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"9","\"Estación\"":"Alcala de Henares","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.5","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Alpedrete","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"9","\"Estación\"":"Aranjuez","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.4","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Arganda del Rey","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"66","MONTH":"3","\"Presión (hPa)\"":"901.4","HOUR":"10","\"Tendencia (hPa)\"":"0.9","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"8","\"Estación\"":"Buitrago del Lozoya","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.4","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"75","MONTH":"3","\"Presión (hPa)\"":"902.3","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"9","\"Estación\"":"Colmenar Viejo","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.3","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"59","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"4","\"Estación\"":"Madrid, Ciudad Universitaria","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.4","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Madrid, El Goloso","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"62","MONTH":"3","\"Presión (hPa)\"":"938.7","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"10","\"Estación\"":"Madrid, Retiro","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.7","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"83","MONTH":"3","\"Presión (hPa)\"":"949.7","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"11","\"Estación\"":"Madrid Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.7","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Pozuelo de Alarcón","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.7","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"68","MONTH":"3","\"Presión (hPa)\"":"846.1","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"26","\"Estación\"":"Puerto Alto del León","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"0.2","\"Velocidad del viento (km/h)\"":"17"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"62","MONTH":"3","\"Presión (hPa)\"":"809.5","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"26","\"Estación\"":"Puerto de Navacerrada","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"0.1","\"Velocidad del viento (km/h)\"":"10"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Rascafría","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"54","MONTH":"3","\"Presión (hPa)\"":"928.8","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"8","\"Estación\"":"Robledo de Chavela","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.8","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Rozas de Puerto Real","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"San Sebastián de Los Reyes","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"57","MONTH":"3","\"Presión (hPa)\"":"853.9","HOUR":"10","\"Tendencia (hPa)\"":"0.7","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Somosierra","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"3.0","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tielmes","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"67","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"9","\"Estación\"":"Torrelodones","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.0","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"76","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"9","\"Estación\"":"Valdemorillo","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.6","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Madrid","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"942.0","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"11","\"Estación\"":"Villanueva de la Cañada","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.5","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Alforja","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"57","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"10","\"Estación\"":"Arenys de Mar","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"15.2","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"88","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Artesa de Segre","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.9","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Balsareny","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"80","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"14","\"Estación\"":"Barcelona","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.3","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Barcelona, Museo Marítimo","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"72","MONTH":"3","\"Presión (hPa)\"":"1015.3","HOUR":"10","\"Tendencia (hPa)\"":"2.0","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"24","\"Estación\"":"Barcelona Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.9","\"Velocidad del viento (km/h)\"":"17"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Berga","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Blanes","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Bossòst","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cabacés","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cabo de Creus","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Caldes de Montbui","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Castell, Platja d'Aro","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Castelló d'Empúries","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Coll de Nargó","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Corbera, Pic d'Agulles","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"El Soleràs","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Espolla","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Esterri d'Àneu","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"54","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"21","\"Estación\"":"Figueres","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"14.0","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"83","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Fogars de Montclús","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.6","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Girona","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"85","MONTH":"3","\"Presión (hPa)\"":"999.3","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"11","\"Estación\"":"Girona Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.2","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Granyanella","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Horta de Sant Joan","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"82","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"18","\"Estación\"":"Igualada","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.4","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"L'Estartit","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"64","MONTH":"3","\"Presión (hPa)\"":"826.1","HOUR":"10","\"Tendencia (hPa)\"":"0.8","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"13","\"Estación\"":"La Molina","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"2.6","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"80","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"La Pobla de Cérvoles","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.6","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"La Pobla de Massaluca","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"82","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"4","\"Estación\"":"La Seu d'Urgell","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.2","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"100","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"6","\"Estación\"":"La Vall de Bianya","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.1","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"100","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"","\"Estación\"":"Les Planes d'Hostoles","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.5","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"78","MONTH":"3","\"Presión (hPa)\"":"993.5","HOUR":"10","\"Tendencia (hPa)\"":"2.0","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"24","\"Estación\"":"Lleida","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.9","\"Velocidad del viento (km/h)\"":"15"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Llimiana","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"92","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"26","\"Estación\"":"Llorac","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.2","\"Velocidad del viento (km/h)\"":"19"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Maçanet de Cabrenys","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"86","MONTH":"3","\"Presión (hPa)\"":"981.5","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"13","\"Estación\"":"Manresa","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.4","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"89","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"5","\"Estación\"":"Martinet","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"2.4","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"96","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"22","\"Estación\"":"Moià","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.6","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Mollerussa","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"98","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Monistrol de Montserrat","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.9","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"87","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"8","\"Estación\"":"Naut Aran, Arties","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"3.7","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"100","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"13","\"Estación\"":"Os de Balaguer","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.9","\"Velocidad del viento (km/h)\"":"8"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"65","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"7","\"Estación\"":"Planoles","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.2","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Pontons","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"77","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"7","\"Estación\"":"Porqueres","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.2","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"86","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"17","\"Estación\"":"Prats de Lluçanès","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.1","\"Velocidad del viento (km/h)\"":"10"},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"61","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"30","\"Estación\"":"Rasquera","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.7","\"Velocidad del viento (km/h)\"":"15"},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"64","MONTH":"3","\"Presión (hPa)\"":"1006.7","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"33","\"Estación\"":"Reus Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.2","\"Velocidad del viento (km/h)\"":"21"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"","\"Estación\"":"Ripoll","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"3.0","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"58","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"14","\"Estación\"":"Santa Susanna","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.5","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Sant Hilari","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"98","MONTH":"3","\"Presión (hPa)\"":"947.7","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"7","\"Estación\"":"Sant Julià  de Vilatorta","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"4.4","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Girona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Sant Pau de Segúries","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Sitges","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"100","MONTH":"3","\"Presión (hPa)\"":"922.3","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"8","\"Estación\"":"Talarn","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"4.3","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tarragona","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"88","MONTH":"3","\"Presión (hPa)\"":"945.1","HOUR":"10","\"Tendencia (hPa)\"":"1.2","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"9","\"Estación\"":"Tona","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.1","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Torà","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"82","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Torre de Cabdella","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"1.3","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"62","MONTH":"3","\"Presión (hPa)\"":"1011.8","HOUR":"10","\"Tendencia (hPa)\"":"1.9","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"31","\"Estación\"":"Tortosa","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.9","\"Velocidad del viento (km/h)\"":"13"},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tuixent","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Lleida","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Vall de Boí","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Tarragona","\"Dirección de racha\"":"","\"Humedad (%)\"":"71","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Valls","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.9","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Vilafranca del Penedès","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Barcelona","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Vilassar de Dalt","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"64","MONTH":"3","\"Presión (hPa)\"":"916.2","HOUR":"10","\"Tendencia (hPa)\"":"1.4","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"10","\"Estación\"":"Abla","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.1","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"49","MONTH":"3","\"Presión (hPa)\"":"1016.6","HOUR":"10","\"Tendencia (hPa)\"":"0.6","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"14","\"Estación\"":"Adra","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"16.1","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Aguilar de la Frontera","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"63","MONTH":"3","\"Presión (hPa)\"":"949.1","HOUR":"10","\"Tendencia (hPa)\"":"1.4","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"5","\"Estación\"":"Alajar","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"13.4","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Albox","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Alcalá La Real","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Algarrobo","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Almadén de la Plata","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"59","MONTH":"3","\"Presión (hPa)\"":"1014.0","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"13","\"Estación\"":"Almería Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"14.9","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"97","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"12","\"Estación\"":"Almonte","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.6","\"Velocidad del viento (km/h)\"":"8"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Álora","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"92","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"14","\"Estación\"":"Alosno, Tharsis","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.6","\"Velocidad del viento (km/h)\"":"8"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Alpandeire","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"99","MONTH":"3","\"Presión (hPa)\"":"992.7","HOUR":"10","\"Tendencia (hPa)\"":"1.1","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"8","\"Estación\"":"Andújar","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.0","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"88","MONTH":"3","\"Presión (hPa)\"":"968.0","HOUR":"10","\"Tendencia (hPa)\"":"1.4","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"7","\"Estación\"":"Antequera","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.9","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Aroche","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"67","MONTH":"3","\"Presión (hPa)\"":"1015.2","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"11","\"Estación\"":"Ayamonte","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"13.2","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"100","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"9","\"Estación\"":"Baeza","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.6","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Barbate","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"97","MONTH":"3","\"Presión (hPa)\"":"926.2","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"10","\"Estación\"":"Baza","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.7","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Benahavís","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Benamejí","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cabo de Gata","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"86","MONTH":"3","\"Presión (hPa)\"":"1016.4","HOUR":"10","\"Tendencia (hPa)\"":"1.4","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"11","\"Estación\"":"Cádiz","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"14.7","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cala","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cardeña","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Carmona","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Carrión de los Céspedes","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cartaya","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Castell de Ferro","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"80","MONTH":"3","\"Presión (hPa)\"":"952.0","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"8","\"Estación\"":"Cazalla de la Sierra","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.9","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cazorla","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Chipiona","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Coín","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"98","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"10","\"Estación\"":"Córdoba, Embalse de Guadanuño","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.5","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"96","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"13","\"Estación\"":"Córdoba, Prágdena","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.0","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"92","MONTH":"3","\"Presión (hPa)\"":"1006.7","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"24","\"Estación\"":"Córdoba Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.7","\"Velocidad del viento (km/h)\"":"12"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"93","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"6","\"Estación\"":"Cortes de la Frontera","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.9","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Dólar","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.9","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"81","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"12","\"Estación\"":"Doña Mencía","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.8","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Écija","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"El Bosque","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"76","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"7","\"Estación\"":"El Campillo","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.8","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"El Cerro de Andévalo","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"El Granado","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Espiel","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"62","MONTH":"3","\"Presión (hPa)\"":"1015.2","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"6","\"Estación\"":"Estepona","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"14.7","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"75","MONTH":"3","\"Presión (hPa)\"":"1014.6","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"19","\"Estación\"":"Fuengirola","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Fuente de Piedra","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Fuente Palmera","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Fuentes de Andalucía","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"75","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"10","\"Estación\"":"Garrucha, Puerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"13.6","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"93","MONTH":"3","\"Presión (hPa)\"":"950.6","HOUR":"10","\"Tendencia (hPa)\"":"1.1","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"5","\"Estación\"":"Granada Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.9","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"80","MONTH":"3","\"Presión (hPa)\"":"936.8","HOUR":"10","\"Tendencia (hPa)\"":"1.0","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"9","\"Estación\"":"Granada Base Aérea","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.7","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Granada-Cartuja","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"72","MONTH":"3","\"Presión (hPa)\"":"911.8","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"10","\"Estación\"":"Grazalema","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.7","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"83","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"10","\"Estación\"":"Guadalcanal","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.5","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"80","MONTH":"3","\"Presión (hPa)\"":"952.3","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"5","\"Estación\"":"Hinojosa del Duque","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.2","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"83","MONTH":"3","\"Presión (hPa)\"":"1014.3","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"10","\"Estación\"":"Huelva, Ronda Este","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.1","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"59","MONTH":"3","\"Presión (hPa)\"":"981.4","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"5","\"Estación\"":"Huércal-Overa","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.9","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"84","MONTH":"3","\"Presión (hPa)\"":"949.1","HOUR":"10","\"Tendencia (hPa)\"":"0.9","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"9","\"Estación\"":"Jaén","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.7","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"92","MONTH":"3","\"Presión (hPa)\"":"1013.3","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"9","\"Estación\"":"Jerez de la Frontera Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.0","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Jimena de la Frontera","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Lanjarón","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"99","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"6","\"Estación\"":"La Puebla de los Infantes","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"9.1","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"La Rambla","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"88","MONTH":"3","\"Presión (hPa)\"":"967.7","HOUR":"10","\"Tendencia (hPa)\"":"1.1","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"9","\"Estación\"":"La Roda de Andalucía","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.3","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Las Cabezas de San Juan","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Láujar de Andarax","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Linares","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Lora de Estepa","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"76","MONTH":"3","\"Presión (hPa)\"":"1011.1","HOUR":"10","\"Tendencia (hPa)\"":"1.2","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"20","\"Estación\"":"Málaga, Centro Meteorológico","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.5","\"Velocidad del viento (km/h)\"":"11"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"50","MONTH":"3","\"Presión (hPa)\"":"1013.0","HOUR":"10","\"Tendencia (hPa)\"":"0.6","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"16","\"Estación\"":"Málaga, Puerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"16.9","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"69","MONTH":"3","\"Presión (hPa)\"":"1015.5","HOUR":"10","\"Tendencia (hPa)\"":"1.4","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"32","\"Estación\"":"Málaga Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"13.7","\"Velocidad del viento (km/h)\"":"19"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Manilva","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"23","\"Estación\"":"Marbella","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"14.2","\"Velocidad del viento (km/h)\"":"10"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Marbella, Puerto","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Medina Sidonia","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"75","MONTH":"3","\"Presión (hPa)\"":"1011.8","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"17","\"Estación\"":"Moguer, El Arenosillo","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"13.4","\"Velocidad del viento (km/h)\"":"9"},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Montoro","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"82","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"7","\"Estación\"":"Morón de la Frontera","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.7","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Motril, Puerto","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Nerja","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Olvera","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Osuna","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Priego de Córdoba","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Rincón de la Victoria","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Ronda Instituto","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"86","MONTH":"3","\"Presión (hPa)\"":"1016.0","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"12","\"Estación\"":"Roquetas de Mar","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"12.4","\"Velocidad del viento (km/h)\"":"8"},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Rota, Base Naval","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Salobreña","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"92","MONTH":"3","\"Presión (hPa)\"":"1012.7","HOUR":"10","\"Tendencia (hPa)\"":"1.3","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"14","\"Estación\"":"San Fernando","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"13.8","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"98","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"6","\"Estación\"":"San José del Valle","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.7","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"San Roque","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"91","MONTH":"3","\"Presión (hPa)\"":"927.8","HOUR":"10","\"Tendencia (hPa)\"":"1.2","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"12","\"Estación\"":"Santa Elena","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"6.9","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"99","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"9","\"Estación\"":"Sevilla, Tablada","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.5","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"97","MONTH":"3","\"Presión (hPa)\"":"1013.1","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"13","\"Estación\"":"Sevilla Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.6","\"Velocidad del viento (km/h)\"":"10"},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tarifa","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Sevilla","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"91","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"9","\"Estación\"":"Tomares, Zaudín","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.3","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Torremolinos","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Torres","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Válor","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Valsequillo","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"","\"Humedad (%)\"":"90","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Valverde del Camino","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"10.1","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Cádiz","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"84","MONTH":"3","\"Presión (hPa)\"":"994.8","HOUR":"10","\"Tendencia (hPa)\"":"1.4","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"14","\"Estación\"":"Vejer de la Frontera","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"13.7","\"Velocidad del viento (km/h)\"":"7"},
{"\"Provincia\"":"Almería","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Vélez Blanco - Topares","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Málaga","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Vélez-Málaga","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Córdoba","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"67","MONTH":"3","\"Presión (hPa)\"":"929.9","HOUR":"10","\"Tendencia (hPa)\"":"1.2","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"13","\"Estación\"":"Villanueva de Córdoba","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"11.1","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Jaén","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Villanueva del Arzobispo","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huelva","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Villarrasa","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Granada","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Víznar","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Ainsa","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Albarracín","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"65","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"27","\"Estación\"":"Alcañiz","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.0","\"Velocidad del viento (km/h)\"":"17"},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"","\"Humedad (%)\"":"78","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Alhama de Aragon","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"4.1","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Almudévar","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Andorra","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Aragüés del Puerto","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Bailo","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Ballobar","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"85","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"10","\"Estación\"":"Barbastro","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.7","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Bello","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Benabarre","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Benasque","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"45","MONTH":"3","\"Presión (hPa)\"":"901.3","HOUR":"10","\"Tendencia (hPa)\"":"0.9","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"30","\"Estación\"":"Bielsa","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.5","\"Velocidad del viento (km/h)\"":"16"},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"63","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudeste","\"Racha (km/h)\"":"6","\"Estación\"":"Biescas","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.1","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"76","MONTH":"3","\"Presión (hPa)\"":"915.4","HOUR":"10","\"Tendencia (hPa)\"":"1.2","YEAR":"2020","\"Dirección del viento\"":"Sur","\"Racha (km/h)\"":"5","\"Estación\"":"Calamocha","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"3.4","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"63","MONTH":"3","\"Presión (hPa)\"":"963.7","HOUR":"10","\"Tendencia (hPa)\"":"1.6","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"24","\"Estación\"":"Calanda","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.6","\"Velocidad del viento (km/h)\"":"15"},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"82","MONTH":"3","\"Presión (hPa)\"":"949.8","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"5","\"Estación\"":"Calatayud","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"3.6","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"73","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"32","\"Estación\"":"Canfranc","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"2.0","\"Velocidad del viento (km/h)\"":"12"},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"Este","\"Humedad (%)\"":"91","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Este","\"Racha (km/h)\"":"5","\"Estación\"":"Capella","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.0","\"Velocidad del viento (km/h)\"":"3"},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"71","MONTH":"3","\"Presión (hPa)\"":"992.2","HOUR":"10","\"Tendencia (hPa)\"":"1.9","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"34","\"Estación\"":"Caspe","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.5","\"Velocidad del viento (km/h)\"":"25"},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Castejon de Valdejasa","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Castellote","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Cedrillas","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"70","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"48","\"Estación\"":"Cerler-Cogulla","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"-4.0","\"Velocidad del viento (km/h)\"":"20"},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"Oeste","\"Humedad (%)\"":"81","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"4","\"Estación\"":"Daroca","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"2.9","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Ejea de los Caballeros","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"74","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"24","\"Estación\"":"Fonfría","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"2.3","\"Velocidad del viento (km/h)\"":"12"},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"Sudoeste","\"Humedad (%)\"":"75","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"16","\"Estación\"":"Fraga","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.4","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Hijar","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Huesca","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"89","MONTH":"3","\"Presión (hPa)\"":"951.1","HOUR":"10","\"Tendencia (hPa)\"":"1.7","YEAR":"2020","\"Dirección del viento\"":"Oeste","\"Racha (km/h)\"":"12","\"Estación\"":"Huesca Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"4.8","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Jabaloyas","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"Sudeste","\"Humedad (%)\"":"59","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Calma","\"Racha (km/h)\"":"12","\"Estación\"":"Jaca","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"3.6","\"Velocidad del viento (km/h)\"":"0"},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"La Almunia de Doña Godina","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Lanaja","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"75","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"10","\"Estación\"":"Leciñena","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.3","\"Velocidad del viento (km/h)\"":"6"},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Montalbán","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"Nordeste","\"Humedad (%)\"":"68","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"9","\"Estación\"":"Mosqueruela","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.6","\"Velocidad del viento (km/h)\"":"5"},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"73","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"33","\"Estación\"":"Muniesa","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"4.6","\"Velocidad del viento (km/h)\"":"17"},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"61","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"9","\"Estación\"":"Panticosa-Petrosos","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"0.2","\"Velocidad del viento (km/h)\"":"2"},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Quinto","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Sabiñánigo","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Santa Eulalia del Campo","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Sariñena, Depósito agua","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Seira","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Sopeira","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"65","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"22","\"Estación\"":"Sos del Rey Católico","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"7.1","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tamarite de Litera","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Tarazona","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"66","MONTH":"3","\"Presión (hPa)\"":"913.3","HOUR":"10","\"Tendencia (hPa)\"":"1.4","YEAR":"2020","\"Dirección del viento\"":"Nordeste","\"Racha (km/h)\"":"9","\"Estación\"":"Teruel","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.5","\"Velocidad del viento (km/h)\"":"4"},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Torla - Depósito","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"Norte","\"Humedad (%)\"":"65","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Norte","\"Racha (km/h)\"":"26","\"Estación\"":"Torla-El Cebollar","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"0.1","\"Velocidad del viento (km/h)\"":"14"},
{"\"Provincia\"":"Teruel","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Valderrobres","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Huesca","\"Dirección de racha\"":"Sur","\"Humedad (%)\"":"75","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"Sudoeste","\"Racha (km/h)\"":"18","\"Estación\"":"Valle de Hecho, Hecho","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"5.0","\"Velocidad del viento (km/h)\"":"9"},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"","\"Humedad (%)\"":"","MONTH":"3","\"Presión (hPa)\"":"","HOUR":"10","\"Tendencia (hPa)\"":"","YEAR":"2020","\"Dirección del viento\"":"","\"Racha (km/h)\"":"","\"Estación\"":"Valmadrid","DAY":"3","\"Precipitación (mm)\"":"","\"Temperatura (ºC)\"":"","\"Velocidad del viento (km/h)\"":""},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"59","MONTH":"3","\"Presión (hPa)\"":"987.4","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"44","\"Estación\"":"Zaragoza, Valdespartera","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.2","\"Velocidad del viento (km/h)\"":"21"},
{"\"Provincia\"":"Zaragoza","\"Dirección de racha\"":"Noroeste","\"Humedad (%)\"":"64","MONTH":"3","\"Presión (hPa)\"":"988.3","HOUR":"10","\"Tendencia (hPa)\"":"1.5","YEAR":"2020","\"Dirección del viento\"":"Noroeste","\"Racha (km/h)\"":"32","\"Estación\"":"Zaragoza Aeropuerto","DAY":"3","\"Precipitación (mm)\"":"0.0","\"Temperatura (ºC)\"":"8.2","\"Velocidad del viento (km/h)\"":"22"}]

In [6]:
new_observation=pd.json_normalize(new_data)
new_observation = new_observation.replace(r'^\s*$', np.nan, regex=True)

In [13]:
province=model_selection(new_observation)

In [15]:
new_clean=process_newobservation(new_observation, scaler_provinces, target_province=province)

In [17]:
distances=check_outliers(new_clean,models_provinces,mapper_provinces,province=province)

Number of anomalies in Madrid: 0:
Number of anomalies in Barcelona: 0:
Number of anomalies in Girona: 0:
Number of anomalies in Lleida: 0:
Number of anomalies in Tarragona: 0:
Number of anomalies in Almería: 0:
Number of anomalies in Huelva: 0:
Number of anomalies in Jaén: 0:
Number of anomalies in Málaga: 0:
Number of anomalies in Granada: 0:
Number of anomalies in Cádiz: 0:
Number of anomalies in Sevilla: 0:
Number of anomalies in Córdoba: 0:
Number of anomalies in Huesca: 0:
Number of anomalies in Teruel: 0:
Number of anomalies in Zaragoza: 0:


C:\Users\Ariadna\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Ariadna\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Ariadna\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Ariadna\AppData\Roaming\Python\

In [18]:
test=[{"Provincia":"Huesca","Dirección de racha":"Sudoeste","Humedad (%)":"61","MONTH":"3","Presión (hPa)":"","HOUR":"0","Tendencia (hPa)":"","YEAR":"2020","Dirección del viento":"Sudoeste","Racha (km/h)":"14","Estación":"Valle de Hecho, Hecho","DAY":"3","Precipitación (mm)":"0.0","Temperatura (ºC)":"8.1","Velocidad del viento (km/h)":"7"}]

In [20]:
a=pd.json_normalize(test)

In [22]:
b = a.replace(r'^\s*$', np.nan, regex=True)